In [0]:
# Creating the Title_df DataFrame from the silver layer table
silver_df = spark.table('az_adb_simbus_training.imdb_project_schema.silver_usecase1_table')
# silver_df.limit(5).display()

In [0]:
from pyspark.sql.functions import col, avg, sum as sum_, year, to_date


#preprocessing removing any null value
silver_df_clean = silver_df.filter((col("genres").isNotNull()) & (col("genres") != "\\N"))
# silver_df_clean.limit(5).display()


In [0]:
# Aggregate average rating and total votes per genre per year
# Selected only two decimal from the averageRating
from pyspark.sql.functions import round
gold_df = silver_df_clean.groupBy("genres", "Year")\
    .agg(
        avg("averageRating").alias("avg_rating"),
        sum_("numVotes").alias("total_votes")
    )

gold_df = gold_df.withColumn("avg_rating", round(col("avg_rating"), 2))
# gold_df.limit(5).display()

In [0]:
%python
gold_df.write.mode("overwrite").option('mergeSchema', True).saveAsTable("imdb_project_schema.gold_layer_usecase1")

In [0]:
%sql
select * from imdb_project_schema.gold_layer_usecase1 limit 5 ;